# Blockctificates project
Team members:
* Rasul Khasyanov 
* Sathyart Mishra Sharma
* Artur Grigorev

### Welcome to the Blocktificates demo!

In [22]:
from json import load, loads
from pprint import PrettyPrinter, pprint

with open('certificates.json', 'r') as f:
    certificates_list = load(f)
    
print("Number of certificates: ", len(certificates_list))
print("Example certificate:")
print(certificates_list[0])

Number of certificates:  7
Example certificate:
{'student_id': 1, 'name': 'Ivan Ivanov', 'date': '2017-12-21', 'grade': '87.7'}


In [23]:
from creator import Certificate

certificates = [Certificate(c['student_id'],
                            c['name'],
                            c['date'],
                            c['grade']) for c in certificates_list]

### Instantiating our class

In [24]:
from certificates import BatchIssuer

issuer = BatchIssuer(certificates, "SK2057")

### Take a look on the Merkle root of the tree just built:

In [25]:
print(issuer.mht_root)

973f6f932b45149c2a2e9dcfb6d99421851ad944


### We publish the Merkle root of our batch of certificates to the Bitcoin testnet with one command: 

In [26]:
issuer.publish()

print(issuer.transaction)

d1dd833da37b3273ab03f35da221582158732d1a4f60d9268423b7150c5ec357


### Let's check this on a Blockchain explorer:

[https://live.blockcypher.com/btc-testnet/](https://live.blockcypher.com/btc-testnet/)

### Now our certificates are ready to distribute!

In [27]:
certificates_to_distribute = issuer.distribute_data()

example_certificate = certificates_to_distribute[list(certificates_to_distribute.keys())[0]]
pprint(example_certificate)

{'certificate': <creator.Certificate object at 0x7f826737ccc0>,
 'hashpath': [('2ceba6f8d5102411d4a7f30ee65197b8dc29da24', 'r'),
              ('2ea802769f8fbd3bc2f4b003c7053172b59b1919', 'r'),
              ('810735e210e649107dfdfa5c05fbc9d86d155876', 'r')],
 'position': 'd1dd833da37b3273ab03f35da221582158732d1a4f60d9268423b7150c5ec357',
 'signature': b"0f\x021\x00\x9af\x87_9\x8f\xbe]'\x9ac\x95\x16\x91\xd7\x91U\xa77"
              b'\x95q\xe9\xf7\x03&\xe3@X4\xd3%\xa1\x18k\xcc\xb5e\xdf\xd2'
              b'\xa6\xa13\x16\xa4\xdd\xec/\xb1\x021\x00\x97=\x88O\x17[\x0f\x8a'
              b'\xa5\x82\xd1[\x92>\x1b\\\xef\x110\x94/\xffr\xd7\xcf\r\xb4\xd3'
              b'\x1d/\xae\xc9N\x0ec\xa7\xb5\xf1\x86\\\xf8V\x19\xbc\xf1\xd0gY'}


### Now we instantiate a validator:

In [28]:
validator = issuer.create_validator()


certificate = example_certificate['certificate']
hashpath = example_certificate['hashpath']
position = example_certificate['position']
signature = example_certificate['signature']

validator.validate(certificate, hashpath, position, signature)

False

### What it really does

In [29]:
# Calculate root
root = validator._calculate_root(certificate, hashpath)
print("Root calculated: {0}".format(root))

# Compare it to the transaction hash
print(validator._check_position(root, position))

# Check signature
print(validator._check_signature(root, signature))

# Check if it was revoked
print(validator._check_revoked(certificate))

Root calculated: 973f6f932b45149c2a2e9dcfb6d99421851ad944
True
True
False


### Plagiarism detected, revoke the certificate!

In [10]:
issuer.revoke(1, "Plagiarism")

print(validator._check_revoked(certificate))

False


### Some more tests:

#### Invalid hashpath:

In [31]:
wrong_hashpath = certificates_to_distribute[list(certificates_to_distribute.keys())[1]]['hashpath']
wrong_root = validator._calculate_root(certificate, wrong_hashpath)

print(validator._check_position(wrong_root, position))

False


#### Invalid signature:

In [32]:
from signatures import generate_keys, sign

private_key, _ = generate_keys()

some_message = "definately not a merkle root"

wrong_signature = sign(some_message, private_key)

print(validator._check_signature(root, wrong_signature))

False
